# Apple stock analysis

In [27]:
import plotly.express as px
import pandas as pd
from pathlib import Path
import numpy as np
from sklearn.linear_model import LinearRegression 
import plotly.graph_objects as go
import joblib

### Importing Data

In [28]:
output_dir = Path("%pwd").resolve().parent.parent
output_data = output_dir / "data/apple_data.csv"
df = pd.read_csv(output_data)
df.head() #or describe() or info()

,row.Date,row.Open,row.High,row.Low,row.Close,row.Adj Close,row.Volume
0,1980-12-12,0.128348,0.128906,0.128348,0.128348,0.100178,469033600
1,1980-12-15,0.122210,0.122210,0.121652,0.121652,0.094952,175884800
2,1980-12-16,0.113281,0.113281,0.112723,0.112723,0.087983,105728000
3,1980-12-17,0.115513,0.116071,0.115513,0.115513,0.090160,86441600
4,1980-12-18,0.118862,0.119420,0.118862,0.118862,0.092774,73449600


## Exploring data

In [29]:
df.isnull().values.any() # it indicates if the df contains any null value. If False, ok. Else, we have to investigate per column and adopt a scenario of data completion.

False

In [30]:
df['row.Date']

0        1980-12-12
1        1980-12-15
2        1980-12-16
3        1980-12-17
4        1980-12-18
            ...    
10463    2022-06-13
10464    2022-06-14
10465    2022-06-15
10466    2022-06-16
10467    2022-06-17
Name: row.Date, Length: 10468, dtype: object

In [31]:
px.line(df, x="row.Date", y='row.Open').update_xaxes(categoryorder='total descending')

## Linear Models for Trend approximation

Let our serie be called $X_t$. An usual decomposition of it is : $X_t=T_t+C_t$, the so-called $\it{Trend-Cycle}$ decomposition. The first method seen here is to hypothetise that the trend component is linear with time : $T_t=\alpha + \beta t$. With a sample, it creates a system, hence the regression as it is is developped in the support document, which details the usuals methods of time series analysis, the foromulas, go check it to understand the non-developed parts.

We, here, have daily datas and in a year we have 52 weeks and so it increases the number of parameters to 54=52 weeks + $\alpha$ + $\beta$. Therefore, we will implement period-12 (monthly) data to have a number of parameters of 14 : $C_t=\sum_{i=1}^{12} \delta_{t,i} S_{t,i}$, as developed in the document.

Finally, the system is : 
$X=A\theta+\epsilon$, where A is the matrix obtained with the columns of time and of the columns that contain the "lighting" coefficients that activate the column corresponding to the period of the line. $\epsilon$ is the residual, $\theta$ has a dimension of 13 according to the constraint developped in the document.

We decided to cut the serie to make it begin in 2010 because the previous data is very different with the latest. We may test this hypothesis later in the docs (long memory tests on the autocorrelation function).

In [32]:
import numpy as np
import time

def vecteur_indicatrice_periodique_slicing(longueur, periode):
    vecteur = np.zeros(longueur, dtype=int)
    vecteur[::periode] = 1
    return vecteur

def vecteur_indicatrice_periodique_comprehension(longueur, periode):
    vecteur = np.array([1 if i % periode == 0 else 0 for i in range(longueur)], dtype=int)
    return vecteur

# Comparaison des performances
taille_vecteur = 10**6
periode = 100

# Méthode avec slicing
debut_slicing = time.time()
resultat_slicing = vecteur_indicatrice_periodique_slicing(taille_vecteur, periode)
fin_slicing = time.time()
temps_slicing = fin_slicing - debut_slicing

# Méthode avec compréhension de liste
debut_comprehension = time.time()
resultat_comprehension = vecteur_indicatrice_periodique_comprehension(taille_vecteur, periode)
fin_comprehension = time.time()
temps_comprehension = fin_comprehension - debut_comprehension

print(f"Temps (slicing) : {temps_slicing:.5f} secondes")
print(f"Temps (compréhension) : {temps_comprehension:.5f} secondes")

Temps (slicing) : 0.00300 secondes
Temps (compréhension) : 0.17297 secondes


In [33]:
def build_A(n_len, p_per):
    #Build the A matrix of dimension (n_len;(1+p_per))
    t = np.linspace(1, n_len[0], num=n_len[0]).reshape((n_len[0],1)) # the 1 in the dim that is for the time
    gam = np.eye(p_per, dtype=int)
    A = np.concatenate([gam for i in range(n_len[0]//p_per + 1)])
    A = np.concatenate([t, A[:n_len[0],]], axis=1)
    # gam=np.array([1 if i%12==1 else 0 for i in range(n)]) # too slow
    return A

In [34]:
df['row.Date'] = pd.to_datetime(df['row.Date'])
df.set_index('row.Date', inplace=True)

We want to filter our df to have only a value per month, a primitive way to do that is to select the first day of the month whovewer it might be a day off which has no value and it is even not in the df. Therefore, we use the isocalendar extension of pandas to considere dates as isodates, which is a norm that consists of dealing with the days by their value in the week (is it a monday or a saturday ?) and this solves our problem.

#### Period = 52 weeks

In [35]:
# Reduce and resample 
# X=df.loc[(df.index >= '2008-09-01') & (df.index <= '2017-09-30') & (df.index.day == 1),'row.Open'], will not work as said above
mask = (df.index >= '2008-10-01') & (df.index <= '2017-09-30') & (df.reset_index()['row.Date'].dt.isocalendar().day == 1)
X=df.loc[mask.to_numpy(),'row.Open'] # the third condition of the mask does not return a numpy object or the loc method needs one, hence the .to_numpy
px.line(X)

Building the matrix could be done in several ways. Building the period indicators vectors separetely with some techniques to build it one time and shift it with slicing or comprenhesion method. Or by concatening eye matrixes of the length of the period with attention to the final cut. This method will be applied, because I think it's the best in complexity of time and space, but someone can test more rigourously and keep the best method.

In [36]:
A = build_A(X.shape, 52)
A

array([[  1.,   1.,   0., ...,   0.,   0.,   0.],
       [  2.,   0.,   1., ...,   0.,   0.,   0.],
       [  3.,   0.,   0., ...,   0.,   0.,   0.],
       ...,
       [423.,   0.,   0., ...,   0.,   0.,   0.],
       [424.,   0.,   0., ...,   0.,   0.,   0.],
       [425.,   0.,   0., ...,   0.,   0.,   0.]])

In [37]:
reg = LinearRegression().fit(A, X)
thet, const = reg.coef_, reg.intercept_
reg.score(A, X)

0.8994156102992804

In [38]:
fig = go.Figure()
fig.add_trace(go.Scatter(x = A[:,0], y = X))
fig.add_trace(go.Scatter(x = A[:,0], y = reg.predict(A)))# + const))

In [39]:
np.dot(A[:,1:],thet[1:])

array([-0.30134149, -0.26024518,  0.09025568,  0.20893188,  0.29340064,
        0.27953673,  0.25638704,  0.57041979,  0.62857944,  0.44497308,
        0.3802705 , -0.10523533, -0.29252715, -0.33013098, -0.56621769,
       -1.35792851, -0.65182747, -0.37010129, -0.44783937, -0.40477395,
       -0.6698334 , -0.51386611, -0.38562181, -0.16461026, -0.22645559,
       -0.31048817, -0.44264574,  0.13599993,  0.4800476 ,  0.60436327,
        1.03412519,  0.87982437,  0.97320204,  0.41144659,  0.23000326,
        0.16940768,  0.08327685, -0.29597898, -0.23282393, -0.12966976,
        0.52361942,  0.03784609,  0.56336651,  0.78915493,  0.4708811 ,
        0.41653565,  0.13294995, -0.45032388, -0.58346396, -0.59847853,
       -0.68666311, -0.30971357, -0.30134149, -0.26024518,  0.09025568,
        0.20893188,  0.29340064,  0.27953673,  0.25638704,  0.57041979,
        0.62857944,  0.44497308,  0.3802705 , -0.10523533, -0.29252715,
       -0.33013098, -0.56621769, -1.35792851, -0.65182747, -0.37

The period is too long, the model cannot regress values for the periodic representation. However, we have a good estimation of the trend and so when we minus reg.predict(A) from X we have also a good estimation of the seasonnal component.

##### Period = 7

In [40]:
# Reduce and resample 
# X=df.loc[(df.index >= '2008-09-01') & (df.index <= '2017-09-30') & (df.index.day == 1),'row.Open'], will not work as said above
mask = (df.index >= '2011-12-01') & (df.index <= '2017-09-30') & (df.reset_index()['row.Date'].dt.isocalendar().day == 1)
X=df.loc[mask.to_numpy(),'row.Open'] # the third condition of the mask does not return a numpy object or the loc method needs one, hence the .to_numpy
px.line(X)

In [41]:
A = build_A(X.shape, 14)
A

array([[  1.,   1.,   0., ...,   0.,   0.,   0.],
       [  2.,   0.,   1., ...,   0.,   0.,   0.],
       [  3.,   0.,   0., ...,   0.,   0.,   0.],
       ...,
       [272.,   0.,   0., ...,   0.,   0.,   0.],
       [273.,   0.,   0., ...,   0.,   0.,   0.],
       [274.,   0.,   0., ...,   0.,   0.,   0.]])

In [42]:
reg = LinearRegression().fit(A, X)
thet, const = reg.coef_, reg.intercept_
reg.score(A, X)

0.666900545015155

In [43]:
fig = go.Figure()
fig.add_trace(go.Scatter(x = A[:,0], y = X))
fig.add_trace(go.Scatter(x = A[:,0], y = reg.predict(A)))# + const))

In [52]:
np.dot(A[:,1:],thet[1:])
fig.add_trace(go.Scatter(x = A[:,0], y = np.dot(A[:,1:],thet[1:]))) # it is the seasonal component.

#### Elastic Net, period = 26

##### Month unite, Period = ? (may do it or not)

Find a way to filter on a month unite, find a good period and fit a linear model and see.

In [ ]:
# Reduce and resample 
# X=df.loc[(df.index >= '2008-09-01') & (df.index <= '2017-09-30') & (df.index.day == 1),'row.Open'], will not work as said above
mask = (df.index >= '2011-12-01') & (df.index <= '2017-09-30') & (df.reset_index()['row.Date'].dt.isocalendar().day == 1)
X=df.loc[mask.to_numpy(),'row.Open'] # the third condition of the mask does not return a numpy object or the loc method needs one, hence the .to_numpy
px.line(X)

In [ ]:
A = build_A(X.shape, 14)
A

array([[  1.,   1.,   0., ...,   0.,   0.,   0.],
       [  2.,   0.,   1., ...,   0.,   0.,   0.],
       [  3.,   0.,   0., ...,   0.,   0.,   0.],
       ...,
       [272.,   0.,   0., ...,   0.,   0.,   0.],
       [273.,   0.,   0., ...,   0.,   0.,   0.],
       [274.,   0.,   0., ...,   0.,   0.,   0.]])

In [ ]:
reg = LinearRegression().fit(A, X)
thet, const = reg.coef_, reg.intercept_
reg.score(A, X)

0.666900545015155

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(x = A[:,0], y = X))
fig.add_trace(go.Scatter(x = A[:,0], y = reg.predict(A)))# + const))

In [ ]:
np.dot(A[:,1:],thet[1:])
fig.add_trace(go.Scatter(x = A[:,0], y = np.dot(A[:,1:],thet[1:]))) # it is the seasonal component.

In [45]:
from sklearn.linear_model import ElasticNet
A = build_A(X.shape, 26)
elas_reg = ElasticNet(l1_ratio=0.2).fit(A, np.log(X))
thet, const = elas_reg.coef_, elas_reg.intercept_
elas_reg.score(A, np.log(X))

0.6563537475405481

In [46]:
fig = go.Figure()
fig.add_trace(go.Scatter(x = A[:,0], y = np.log(X)))
fig.add_trace(go.Scatter(x = A[:,0], y = elas_reg.predict(A)))# + const))
# fig.add_trace(go.Scatter(x = A[:,0], y = np.dot(A[:,1:],thet[1:]))) it is the seasonal component.

In [47]:
# Cell to add in the script.py file to export the linear model after training it :
# output_path = Path(__file__).resolve().parent / 'model' / 'lin_reg_model.pkl'
# joblib.dump(reg, output_path)

We see that with logarithm model, the $\mathcal(L)^2$ regression is quite bad. We could push further, as in the classical model, by reducing the data (we see that there are inflexions in the beginning of the datas) or change the period (for weekly or other things) but we will only try other regressions models and switch to other methods (in reality the curve isn't linear but we are here to investigate the different methods).

To conclude the linear model as expected is quite bad at this task, we may try to change to a monthly frequency and try to find a better period but we would spend more time on a method which is, as we know, not quite adapted. However, the next one, Moving Average, is way better as one of its purposes was to adress this task.

## Deseasonalysing with the moving average method

In [48]:
a = np.linspace(5, 10, 11)
print(a[0])

5.0


## LGBM Models' Training

### Hyperopt

#### LGBM_hopt_1

In [49]:
from hyperopt import hp, fmin, tpe, Trials, STATUS_OK, space_eval

list_obj = ['tweedie','mae','regression']

lgb_reg_params = { ##définition perso des valeurs possibles
    'objective' :       'regression',
    #'metric':           hp.choice('metric', list_obj), #‘perso_pic’
    'learning_rate':    hp.uniform('learning_rate',0.001,0.5), # ou ,0.1,1)
    #'num_iterations':   hp.choice('num_iterations',       np.arange(5, 200, 1, dtype=int)),
    'num_leaves':       hp.choice('num_leaves',       np.arange(2, 89, 1, dtype=int)),
    'max_depth':        hp.choice('max_depth',        np.arange(2, 100, 1, dtype=int)),
    #'min_child_weight': hp.choice('min_child_weight', np.arange(1, 50, 1, dtype=int)),
    #'colsample_bytree': hp.uniform('colsample_bytree',0.4,1),
    #'subsample':        hp.uniform('subsample', 0.6, 1),
    #'min_split_gain':   hp.uniform('min_split_gain', 0, 1),
}
# lgb_reg_params_comp = {
#     'objective' :       hp.choice('objective', ['tweedie','mae','regression']), #‘perso_pic’
#     'learning_rate':    hp.uniform('learning_rate',0.1,1),
#     'num_leaves':       hp.choice('num_leaves',       np.arange(2, 200, 1, dtype=int)),
#     'max_depth':        hp.choice('max_depth',        np.arange(2, 100, 1, dtype=int)),
#     'min_child_weight': hp.choice('min_child_weight', np.arange(1, 50, 1, dtype=int)),
#     'colsample_bytree': hp.uniform('colsample_bytree',0.4,1),
#     'subsample':        hp.uniform('subsample', 0.6, 1),
#     'min_split_gain':   hp.uniform('min_split_gain', 0, 1),
#     'n_estimators':     5
# } 
##### mettre les espaces des paramètres compris dans Param_LGBM
def f(params) : 
    lgbm = lgb.LGBMRegressor(n_jobs=-1,early_stopping_rounds=None,**params)
    score = cross_val_score(lgbm, X_train, y_train, cv=2,scoring=mae_scorer,n_jobs=-1).mean() ## ['regression', 'tweedie', 'mape'] je peux les mettre ici sinon en utilant sklearn...cross_validate
    return score

In [50]:
trials = Trials()
result = fmin(
    fn=f,                           # objective function
    space=lgb_reg_params,   # parameter space
    algo=tpe.suggest,               # surrogate algorithm
    max_evals=500,                  # no. of evaluations
    trials=trials,                   # trials object that keeps track of the sample results (optional)
    verbose=1
)
result
result = space_eval(lgb_reg_params, result)

  0%|          | 0/500 [00:00<?, ?trial/s, best loss=?]

job exception: name 'lgb' is not defined



  0%|          | 0/500 [00:00<?, ?trial/s, best loss=?]


NameError: name 'lgb' is not defined

In [ ]:
result

In [ ]:
params = result.copy()
#params['objective']='tweedie'
params['num_iterations']=200

lgb_hopt = lgb.LGBMRegressor(**params)
lgb_hopt.fit(X_train, y_train, eval_set=[(X_test, y_test), (X_train, y_train)], verbose=0)
lgb.plot_metric(lgb_hopt)

### Test Plot

In [ ]:
fig = go.Figure()

fig.add_trace(go.Scatter(name="Vraies valeurs", x=y_test.index, y=y_test, mode='lines+markers'))
fig.add_trace(go.Scatter(name="LGBM", x=y_test.index, y=model.predict(X_test).astype(int), mode='lines+markers'))
fig.add_trace(go.Scatter(name="LGBM_HP", x=y_test.index, y=lgb_hopt.predict(X_test).astype(int), mode='lines+markers'))
#fig.add_trace(go.Scatter(name="LGBM_HP_2", x=y_test.index, y=lgb_hopt_2.predict(X_test).astype(int), mode='lines+markers'))


mean=(model.predict(X_test)+lgb_hopt_2.predict(X_test))/2
#fig.add_trace(go.Scatter(name="LGBM_HP_mean", x=y_test.index, y=mean.astype(int), mode='lines+markers'))

fig.show()

#### Plot Aver. Perc. Err.

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(name="LGBM", x=X_test.index, y=predict_test(model, pd.DataFrame(), X_test, y_test)["absolute_error_%"], mode='lines+markers'))

#### Learning & Test Error

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(name="Vraies valeurs", x=pd.concat([X_train, X_test]).index, y=pd.concat([y_train, y_test]), mode='lines+markers'))
fig.add_trace(go.Scatter(name="LGBM", x=X_train.index, y=model.predict(X_train), mode='lines+markers', marker_color='springgreen'))
fig.add_trace(go.Scatter(name="LGBM_hopt", x=X_train.index, y=lgb_hopt.predict(X_train), mode='lines+markers', marker_color='indianred'))

fig.add_trace(go.Scatter(name="LGBM_test", x=X_test.index, y=model.predict(X_test).astype(int), mode='lines+markers', marker_color='#00CC96'))
fig.add_trace(go.Scatter(name="LGBM_hopt_test", x=X_test.index, y=lgb_hopt.predict(X_test).astype(int), mode='lines+markers',  marker_color='#EF553B'))
fig.show()

In [ ]:
tabl_err_all([model, lgb_hopt, lgb_hopt_2], X_train, y_train, X_test, y_test)

In [ ]:
tabl_err_all([model, lgb_hopt, lgb_hopt_2], X_train, y_train, X_test.loc['2023-01-01':'2023-06-09'], y_test.loc['2023-01-01':'2023-06-09']) 

In [ ]:
import joblib
# save model
joblib.dump(lgb_hopt, 'lgbm_reg_D_mail_invoicing.pkl')
# load model
gbm_pickle = joblib.load('lgbm_reg_D_mail_invoicing.pkl')